In [1]:
!{__import__('sys').executable} -m pip install --quiet --upgrade -r requirements.txt

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.6.0 requires numpy~=1.19.2, but you have numpy 1.22.4 which is incompatible.


In [2]:
session_storage = 'downloads'

import os
os.makedirs(session_storage, exist_ok=True)

In [3]:
import fsspec

fs_dataset = fsspec.filesystem(
    'simplecache', 
    target_protocol='https', 
    cache_storage=session_storage,
)

In [4]:
import pandas as pd
'''
import swifter
swifter.set_defaults(
    npartitions=None,
    dask_thres=1,
    scheduler='processes',
    progress_bar=True,
    progress_bar_desc=None,
    allow_dask_on_strings=True,
    force_parallel=False
)
'''

import sklearn as skl
import sklearn.base
import sklearn.compose
#import sklearn.ensemble
import sklearn.feature_extraction
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline
import sklearn.svm

import xgboost

import numpy as np

In [5]:
import sklearnex
sklearnex.patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Dataset

See https://ftp.funet.fi/pub/mirrors/ftp.imdb.com/pub/frozendata

In [6]:
import re

def _parse_imdb_list(fo):
    pattern = re.compile(
        fr'^(?P<key>[^:]+):\s+(?P<value>.*)$', 
        flags=re.DOTALL
    )
    for l in fo.read().splitlines():
        m = re.match(pattern, l)
        if not m:
            continue
        yield m.groupdict()

def _parse_imdb_list_plot(f):
    r = None

    for entry in _parse_imdb_list(f):
        # 
        if entry['key'] == 'MV':
            if r is not None:
                yield r

            r = dict()
            r['name'] = entry['value']

        if r is None:
            # error
            continue

        if entry['key'] == 'BY':
            r.setdefault('plot_authors', [])
            r['plot_authors'].append(entry['value'])

        if entry['key'] == 'PL':
            r.setdefault('plot_text', [])
            r['plot_text'].append(entry['value'])

In [7]:
dataset_plots = None
with fs_dataset.open(
    'https://ftp.funet.fi/pub/mirrors/ftp.imdb.com/pub/frozendata/plot.list.gz', 
    compression='gzip', 
    mode='rt', errors='ignore'
) as f:
    dataset_plots = pd.DataFrame(
        list(_parse_imdb_list_plot(f))
    )

In [8]:
dataset_plots

,name,plot_text,plot_authors
0,"""#7DaysLater"" (2013)",[#7dayslater is an interactive comedy series f...,[Sum Whan]
1,"""#BlackLove"" (2015) {Bringing Sexy Back (#1.3)}","[This week, the five women work on getting wha...",NaN
2,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","[With just one week left in the workshops, the...",NaN
3,"""#BlackLove"" (2015) {Like a Virgin (#1.4)}",[All of the women are struggling with what mak...,NaN
4,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",[All of the women start making strides towards...,NaN
...,...,...,...
596421,deux (2017),[When 2 Musicians throw coins in the same wish...,[Randy Abbassi]
596422,propos: philosophie (2016),[Is happiness good for human beings? Is truth ...,[Sebastian Kthe]
596423,a gote le ciel (2014),[The 'Sky's the Limit' is the story of an 11 y...,[Anonymous]
596424,solo un nastro che gira (2017),[A woman in crisis finds in a peculiar bar the...,NaN


In [19]:
dataset_plots['plot_text'].str.len().describe()

count    596426.000000
mean          8.835435
std           6.687068
min           1.000000
25%           5.000000
50%           7.000000
75%          10.000000
max         273.000000
Name: plot_text, dtype: float64

In [15]:
dataset_plots.iloc[5000]['plot_text']

['The Escapee presents a woman on the verge of abandoning her upcoming',
 'marriage and perhaps a way of life as she inspects what is important in her',
 'life and what is merely the dictate of convention providing us with an',
 'insight to her unique way of looking at things.']

In [9]:
dataset_plots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596426 entries, 0 to 596425
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   name          596426 non-null  object
 1   plot_text     596426 non-null  object
 2   plot_authors  506486 non-null  object
dtypes: object(3)
memory usage: 13.7+ MB
